Reference : https://github.com/mrdbourke/pytorch-deep-learning/blob/main/03_pytorch_computer_vision.ipynb


https://www.learnpytorch.io/03_pytorch_computer_vision/

# PyTorch module

# torchvision:	
Contains datasets, model architectures and image transformations often used for computer vision problems.


# torchvision.datasets:	
Here you'll find many example computer vision datasets for a range of problems from image classification, object detection, image captioning, video classification and more. It also contains a series of base classes for making custom datasets.
# torchvision.models:	
This module contains well-performing and commonly used computer vision model architectures implemented in PyTorch, you can use these with your own problems.
# torchvision.transforms:	
Often images need to be transformed (turned into numbers/processed/augmented) before being used with a model, common image transformations are found here.
# torch.utils.data.Dataset:	
Base dataset class for PyTorch.
#torch.utils.data.DataLoader:	
Creates a Python iteralbe over a dataset (created with torch.utils.data.Dataset).

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt



# 1. Getting a dataset
To begin working on a computer vision problem, let's get a computer vision dataset.

We're going to start with FashionMNIST.

MNIST stands for Modified National Institute of Standards and Technology.

The original MNIST dataset contains thousands of examples of handwritten digits (from 0 to 9) and was used to build computer vision models to identify numbers for postal services.

FashionMNIST, made by Zalando Research, is a similar setup.

Except it contains grayscale images of 10 different kinds of clothing.

* root: str - which folder do you want to download the data to?
* train: Bool - do you want the training or test split?
* download: Bool - should the data be downloaded?
* transform: torchvision.transforms - what transformations would you like to do on the data?
* target_transform - you can transform the targets (labels) if you like too

In [ ]:
from torchvision.datasets.fakedata import transforms
#setup traning data
train_data = datasets.FashionMNIST(
    root='dahwin',
    train=True,
    download=True,
    transform = ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root='dahwin',
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
# see first training sample
image,label = train_data[0]
# image,label


In [ ]:
image.shape

In [ ]:
len(train_data.data),len(train_data.targets),len(test_data.data),len(test_data.targets)

In [ ]:
class_names= train_data.classes

# 1.2 Visualizing our data

In [ ]:
print(f"Image shape: {image.shape}")
plt.imshow(image.squeeze()) # image shape is [1, 28, 28] (colour channels, height, width)
plt.title(label);

In [ ]:
plt.imshow(image.squeeze(),cmap='gray')

In [ ]:
# ploting more images
torch.manual_seed(42)
fig = plt.figure(figsize=(9,30))
rows,cols = 10,4
for i in range(1,rows*cols+1):
  random_idx = torch.randint(0,len(train_data),size=[1]).item()
  img,label = train_data[random_idx]
  fig.add_subplot(rows,cols,i)
  plt.imshow(img.squeeze(),cmap='gray')
  plt.title(class_names[label])
  plt.axis(False)

# 2. Prepare DataLoader
Now we've got a dataset ready to go.

The next step is to prepare it with a torch.utils.data.DataLoader or DataLoader for short.

The DataLoader does what you think it might do.

It helps load data into a model.

For training and for inference.

It turns a large Dataset into a Python iterable of smaller chunks.

These smaller chunks are called batches or mini-batches and can be set by the batch_size parameter.

Why do this?

Because it's more computationally efficient.

In an ideal world you could do the forward pass and backward pass across all of your data at once.

But once you start using really large datasets, unless you've got infinite computing power, it's easier to break them up into batches.

It also gives your model more opportunities to improve.

With mini-batches (small portions of the data), gradient descent is performed more often per epoch (once per mini-batch rather than once per epoch).

What's a good batch size?

32 is a good place to start for a fair amount of problems.

But since this is a value you can set (a hyperparameter) you can try all different kinds of values, though generally powers of 2 are used most often (e.g. 32, 64, 128, 256, 512).

In [ ]:
from torch.utils.data import DataLoader
batch_size = 32
train_dataloader = DataLoader( train_data,
                              batch_size=batch_size,
                              shuffle=True
    
)
test_dataloader = DataLoader(test_data,
                             batch_size=batch_size,
                             shuffle=False)
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {batch_size}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {batch_size}")

In [ ]:
train_features_batch ,train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape,train_labels_batch.shape

In [ ]:
torch.manual_seed(42)
random_idx = torch.randint(0,len(train_features_batch),size=[1]).item()
img,label = train_features_batch[random_idx],train_labels_batch[random_idx]
plt.imshow(img.squeeze())
plt.title(class_names[label])
plt.axis('Off')
print(f'Image size: {img.shape}')
print(f'Label:{label},label size: {label.shape}')

# 3. Model 0: Build a baseline model
Data loaded and prepared!

Time to build a baseline model by subclassing nn.Module.

A baseline model is one of the simplest models you can imagine.

You use the baseline as a starting point and try to improve upon it with subsequent, more complicated models.

Our baseline will consist of two nn.Linear() layers.

We've done this in a previous section but there's going to one slight difference.

Because we're working with image data, we're going to use a different layer to start things off.

And that's the nn.Flatten() layer.

nn.Flatten() compresses the dimensions of a tensor into a single vector.

This is easier to understand when you see

In [ ]:
# create a flattttten layer
flatten_model = nn.Flatten()
# Get a single sample
x = train_features_batch[0]

output = flatten_model(x)
# Print out what happened
print(f"Shape before flattening: {x.shape} -> [color_channels, height, width]")
print(f"Shape after flattening: {output.shape} -> [color_channels, height*width]")

In [ ]:
from torch import nn
class DahwinFashionMNISTModelV0(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
      super().__init__()
      self.layer_stack = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=input_shape,out_features=hidden_units),
          nn.Linear(in_features=hidden_units,out_features=output_shape))
  def forward(self,x):
    return self.layer_stack(x)
   

In [ ]:
torch.manual_seed(42)
model_0 = DahwinFashionMNISTModelV0(input_shape=784,
                                    hidden_units=10,output_shape=len(class_names))
model_0

# setup loss,optimizer and evaluation metrics 
since we're working on a classification problem,let's bring in our helper_fuctions.py script and 
subsquentlyl the accuracy_fn() we defined in notebook

In [ ]:
import requests
from pathlib import Path
if Path('helper_fuctioins.py').is_file():
  print('already exists')
else:
  print('Downloading')
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params= model_0.parameters(),lr=0.1)

In [ ]:
import time

# 3.2 Creating a function to time our experiments
Loss function and optimizer ready!

It's time to start training a model.

But how about we do a little experiment while we train.

I mean, let's make a timing function to measure the time it takes our model to train on CPU versus using a GPU.

We'll train this model on the CPU but the next one on the GPU and see what happens.

Our timing function will import the timeit.default_timer() function from the Python timeit module. 

# 3.3 Creating a training loop and training a model on batches of data
Beautiful!

Looks like we've got all of the pieces of the puzzle ready to go, a timer, a loss function, an optimizer, a model and most importantly, some data.

Let's now create a training loop and a testing loop to train and evaluate our model.

We'll be using the same steps as the previous notebook(s), though since our data is now in batch form, we'll add another loop to loop through our data batches.

Our data batches are contained within our DataLoaders, train_dataloader and test_dataloader for the training and test data splits respectively.

A batch is BATCH_SIZE samples of X (features) and y (labels), since we're using BATCH_SIZE=32, our batches have 32 samples of images and targets.

And since we're computing on batches of data, our loss and evaluation metrics will be calculated per batch rather than across the whole dataset.

This means we'll have to divide our loss and accuracy values by the number of batches in each dataset's respective dataloader.

Let's step through it:

Loop through epochs.
Loop through training batches, perform training steps, calculate the train loss per batch.
Loop through testing batches, perform testing steps, calculate the test loss per batch.
Print out what's happening.
Time it all (for fun).
A fair few steps but...

In [ ]:
from tqdm.auto import tqdm
torch.manual_seed(42)
start = time.time()
epochs =3
for epoch in tqdm(range(epochs)):
  print(f"Epoch:{epoch}\n....")
  train_loss = 0
  # Add a loop to loop through training batches
  for batch ,(x,y) in enumerate(train_dataloader):
    model_0.train()
    y_pre = model_0(x)
    loss = loss_fn(y_pre,y)
    # calculate loss (per batch)
    train_loss += loss # accumulatively add up the loss per epoch
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 400 ==0:
      print(f'LOoked at {batch*len(x)}/{len(train_dataloader.dataset)} samples')
  # Divide total train loss by lenth of train dataloader
  train_loss /= len(train_dataloader)
  ### testing 
  test_loss,test_acc = 0,0
  model_0.eval()
  with torch.inference_mode():
    for x_test,y_test in test_dataloader:
      test_pre = model_0(x_test)
      # calculate loss (accumulatively)
      test_loss = loss_fn(test_pre,y_test)
      test_acc += accuracy_fn(y_test,test_pre.argmax(dim=1))
    # calculate the loss avarage per batch
    test_loss /= len(test_dataloader)
    # calculate the test acc avarage per batch
    test_acc /= len(test_dataloader)
  print(f"\n Train loss:{train_loss:.4f}| test loss: {test_loss:.4f},Test acc: {test_acc:.4f}")
  # calculat training time
  end = time.time()
  time0 = end-start
  print(time0)
  






# 4. Make predictions and get Model 0 results

In [ ]:
torch.manual_seed(42)
def eval_model(model:torch.nn.Module,
         data_loader:torch.utils.data.DataLoader,
         loss_fn:torch.nn.Module,
         accuracy_fn):
    loss ,acc = 0 , 0
    model.eval()
    with torch.inference_mode():
      for x,y in data_loader:
       
        y_pre = model(x)

        #Accumulate the loss and acc values per batch
        loss += loss_fn(y_pre,y)
        acc += accuracy_fn(y,y_pre.argmax(dim=1))
    # scale loss and acc to find the averagae loss/acc per batch
      loss /= len(data_loader)
      acc /= len(data_loader)
    return {
        'model_name': model.__class__.__name__,
        'model_loss': loss.item(),
        'model_acc': acc
    }
# calculate model 0 results on test dataset
model_0_results = eval_model(
    model=model_0,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    
)
model_0_results

# 5. Setup device agnostic-code (for using a GPU if there is one)

In [ ]:
# Setup device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Model1: Building a better model with non-linearity


In [ ]:
class DahwinFashionMNISTModelV1(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,out_features=output_shape),
        nn.ReLU()
    )
  def forward(self,x:torch.Tensor):
  
   return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)
model_1 = DahwinFashionMNISTModelV1(input_shape=784,hidden_units=10,output_shape=len(class_names)).to(device)
next(model_1.parameters()).device

In [ ]:
# 6.1 Setup los,optimiiiizer and evaluation metrics
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),lr=0.1)

# 6.2 Functionizing traning and evaluation loops
Let's create a fuction for 
* training loop 
* testing loop

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (x, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = x.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(
              model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for x, y in data_loader:
            # Send data to GPU
            x, y = x.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(x)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
torch.manual_seed(42)
start = time.time()
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch:{epoch}\n........")
  train_step(model_1,train_dataloader,loss_fn,optimizer,accuracy_fn,device)
  test_step(model_1,test_dataloader,loss_fn,accuracy_fn,device)
end = time.time()
time1 = end-start
print(time1)

# Note: The training time on CUDA vs CPU will depend largely on the quality of the CPU/GPU you're using. Read on for a more explained answer.

Question: "I used a a GPU but my model didn't train faster, why might that be?"

Answer: Well, one reason could be because your dataset and model are both so small (like the dataset and model we're working with) the benefits of using a GPU are outweighed by the time it actually takes to transfer the data there.

There's a small bottleneck between copying data from the CPU memory (default) to the GPU memory.

So for smaller models and datasets, the CPU might actually be the optimal place to compute on.

But for larger datasets and models, the speed of computing the GPU can offer usually far outweighs the cost of getting the data there.

However, this is largely dependant on the hardware you're using. With practice, you will get used to where the best place to train your models i

In [ ]:
# Move values to device
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, 
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        
        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

# Calculate model 1 results with device-agnostic code 
model_1_results = eval_model(
    model=model_1,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    
)
model_1_results

# 7. Model 2 : Building a Convolutional Neural Network (CNN)

In [ ]:
from torch.nn.modules import Conv2d
class DahwinFashionMNISTModelCNNV2(nn.Module):
  def __init__(self,input_shape:int,hidden_layers:int,output_shape:int):
    super().__init__()
    self.conv_block = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_layers,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_layers,out_channels=hidden_layers,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_layers,out_channels=hidden_layers,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_layers,out_channels=hidden_layers,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_layers*7*7,out_features=output_shape)
    )
  def forward(self,x):
    x = self.conv_block(x)
    # print(f"output shape of conv_block1:{x.shape}")
    x = self.conv_block_2(x)
    # print(f"output shape of conv_block2:{x.shape}")

    x = self.classifier(x)
    # print(f"output shape of classifier:{x.shape}")
    return x
  

In [ ]:
image.shape

In [ ]:
torch.manual_seed(42)
model_2 = DahwinFashionMNISTModelCNNV2(input_shape=1,hidden_layers=10,output_shape=len(class_names)).to(device)
model_2

In [ ]:
plt.imshow(image.squeeze())

In [ ]:
random_image_tensor =torch.randn(1,28,28)

In [ ]:
model_2(image.unsqueeze(0).to(device))

# 7.1 Stepping through nn.Conv2d

In [ ]:
torch.manual_seed(42)
# create a batch of images
images = torch.randn(size=(32,3,64,64))
test_images = images[0]
print(f"Image batch shape:{images.shape}")
print(f"single image shape:{test_images.shape}")
print(f"Test image:\n{test_images}")

In [ ]:
torch.manual_seed(42)

# Create a convolutional layer with same dimensions as TinyVGG 
# (try changing any of the parameters and see what happens)
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=3,
                       stride=1,
                       padding=0) # also try using "valid" or "same" here 

# Pass the data through the convolutional layer
conv_layer(test_images) # Note: If running PyTorch <1.11.0, this will error because of shape issues (nn.Conv.2d() expects a 4d tensor as input) 


In [ ]:
test_images.shape

In [ ]:
test_images.unsqueeze(dim=0).shape

In [ ]:
conv_layer(test_images.unsqueeze(dim=0)).shape

In [ ]:
torch.manual_seed(42)
# Create a new conv_layer with different values (try setting these to whatever you like)
conv_layer_2 = nn.Conv2d(in_channels=3, # same number of color channels as our input image
                         out_channels=64,
                         kernel_size=(3, 3), # kernel is usually a square so a tuple also works
                         stride=1,
                         padding=1)

# Pass single image through new conv_layer_2 (this calls nn.Conv2d()'s forward() method on the input)
conv_layer_2(test_images.unsqueeze(dim=0)).shape

In [ ]:
# Check out the conv_layer_2 internal parameters
print(conv_layer_2.state_dict())

In [ ]:
# Get shapes of weight and bias tensors within conv_layer_2
print(f"conv_layer_2 weight shape: \n{conv_layer_2.weight.shape} -> [out_channels=10, in_channels=3, kernel_size=5, kernel_size=5]")
print(f"\nconv_layer_2 bias shape: \n{conv_layer_2.bias.shape} -> [out_channels=10]")

7.2 stepiing through nn.MaxPool2d()

In [ ]:

print(f"test image original shape{test_images.shape}")
print(f"test image with unsqueezed dimension:{test_images.unsqueeze(0).shape}")
# create a sample of nn.MaxPool2d layer
max_pool2d_layer = nn.MaxPool2d(kernel_size=2)
# pass data through just the conv_layer
test_image_through_conv = conv_layer(test_images.unsqueeze(dim=0))
print(f"shape after going through conv_layer():{test_image_through_conv.shape}")
# pass data through the max pool layer

test_image_through_conv_max_pool = max_pool2d_layer(test_image_through_conv)
print(f"shape after going through conv_layer() and maxpool2d() layer: {test_image_through_conv_max_pool.shape}")

In [ ]:
torch.manual_seed(42)
# create random tensor with a similer number of dimensions to our images
random_tensor = torch.randn(1,1,2,2)
print(f"random tensor:{random_tensor}")
print(f"random tensor shape:{random_tensor.shape}")
# create a maxpool layer
max_pool2d_layer = nn.MaxPool2d(2)
max_pool_tensor = max_pool2d_layer(random_tensor)
print(f"\nMax pool tensor:\n {max_pool_tensor}")
print(f"Max pool tesnor shape: {max_pool_tensor.shape}")

# 7.3 setup a loss fuction and optimizer for model_2

In [ ]:
# setup loss fuction /eval metrics/optimizer
from helper_functions import accuracy_fn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)


# 7.4 training and testing model_2 using our training and test functions

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
start = time.time()
epochs = 3
for epoch in tqdm(range(epochs)):
  print(f"Epoch:{epoch}\n.....")
  train_step(model_2,train_dataloader,loss_fn,optimizer,accuracy_fn,device)
  test_step(model_2,test_dataloader,loss_fn,accuracy_fn,device)

end = time.time()
time2 = end-start
print(time2)


In [ ]:
model_2_results = eval_model(
    model=model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    
)
model_2_results


# 8. Compare model results and training time

In [ ]:
import pandas

In [ ]:
import pandas as pd
compare_results = pd.DataFrame([model_0_results, model_1_results, model_2_results])
compare_results

In [ ]:
# visualize our model results
compare_results.set_index('model_name') ['model_acc'].plot(kind='barh')
plt.xlabel('accuracy(%)')
plt.ylabel('model')

In [ ]:
compare_results['executing time'] = [time0,time1,time2]
compare_results

In [ ]:
def make_predictions(model:torch.nn.Module,data:list,device:torch.device=device):
    pre =[]
    model.eval()
    with torch.inference_mode():
      for sample in data:
        sample = torch.unsqueeze(sample,dim=0).to(device)
        pre_logits = model(sample)
        pr = torch.softmax(pre_logits.squeeze(),dim=0)
        pre.append(pr.cpu())

    return torch.stack(pre)

In [ ]:
import random
random.seed(42)
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_data), k=9):
    test_samples.append(sample)
    test_labels.append(label)

# View the first test sample shape and label
print(f"Test sample image shape: {test_samples[0].shape}\nTest sample label: {test_labels[0]} ({class_names[test_labels[0]]})")


In [ ]:
# Make predictions on test samples with model 2
pred_probs= make_predictions(model=model_2, 
                             data=test_samples)

# View first two prediction probabilities list
pred_probs[:2]

In [ ]:
# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
# Are our predictions in the same form as our test labels? 
test_labels, pred_classes

In [ ]:
# Plot predictions
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
  # Create a subplot
  plt.subplot(nrows, ncols, i+1)

  # Plot the target image
  plt.imshow(sample.squeeze(), cmap="gray")

  # Find the prediction label (in text form, e.g. "Sandal")
  pred_label = class_names[pred_classes[i]]

  # Get the truth label (in text form, e.g. "T-shirt")
  truth_label = class_names[test_labels[i]] 

  # Create the title text of the plot
  title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
  if pred_label == truth_label:
      plt.title(title_text, fontsize=10, c="g") # green text if correct
  else:
      plt.title(title_text, fontsize=10, c="r") # red text if wrong
  plt.axis(False);

# Make all test prediction

In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# 1. Make predictions with trained model
y_preds = []
model_2.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions"):
    # Send data and targets to target device
    X, y = X.to(device), y.to(device)
    # Do the forward pass
    y_logit = model_2(X)
    # Turn predictions from logits -> prediction probabilities -> predictions labels
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor

# Making confusion matrixcs

In [ ]:

# See if torchmetrics exists, if not, install it
try:
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")
    assert int(mlxtend.__version__.split(".")[1]) >= 19, "mlxtend verison should be 0.19.0 or higher"
except:
    !pip install -q torchmetrics -U mlxtend # <- Note: If you're using Google Colab, this may require restarting the runtime
    import torchmetrics, mlxtend
    print(f"mlxtend version: {mlxtend.__version__}")

In [ ]:
# Import mlxtend upgraded version
import mlxtend 
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # should be version 0.19.0 or higher

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion matrix instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=len(class_names), task='multiclass')
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_data.targets)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with NumPy 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

In [ ]:
from pathlib import Path

# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)

# Create model save path
MODEL_NAME = "DahwinFashionMNISTModelCNNV2.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_2.state_dict(), # only saving the state_dict() only saves the learned parameters
           f=MODEL_SAVE_PATH)

In [ ]:

# Create a new instance of FashionMNISTModelV2 (the same class as our saved state_dict())
# Note: loading model will error if the shapes here aren't the same as the saved version
loaded_model_2 = DahwinFashionMNISTModelCNNV2(input_shape=1, 
                                    hidden_layers=10, # try changing this to 128 and seeing what happens 
                                    output_shape=10) 

# Load in the saved state_dict()
loaded_model_2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

# Send model to GPU
loaded_model_2 = loaded_model_2.to(device)

In [ ]:
# Evaluate loaded model
torch.manual_seed(42)

loaded_model_2_results = eval_model(
    model=loaded_model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn, 
    accuracy_fn=accuracy_fn
)

loaded_model_2_results

In [ ]:
model_2_results

In [ ]:
# Check to see if results are close to each other (if they are very far away, there may be an error)
torch.isclose(torch.tensor(model_2_results["model_loss"]), 
              torch.tensor(loaded_model_2_results["model_loss"]),
              atol=1e-08, # absolute tolerance
              rtol=0.0001) # relative tolerance